<img src="images/dask_horizontal.svg" align="right" width="30%">

# 分布式

到目前为止，我们已经看到，Dask 允许您简单地构建具有依赖关系的任务图，以及在数据集合上使用函数式、Numpy 或 Pandas 语法为您自动创建图。如果没有以并行和内存感知方式执行这些图的方法，那么这些都不会非常有用。到目前为止，我们一直在调用 `thing.compute()` 或 `dask.compute(thing)` 而不必担心这意味着什么。现在我们将讨论可用于该执行的选项，特别是带有附加功能的分布式调度器(`scheduler`)。

Dask有四个可用的调度器：
- "线程"(`threaded`)（又名`threading`）：由线程池支持的调度器
- "进程"(`processes`)：由进程池支持的调度器
- "单线程"(`single-threaded`)（又名"同步"(`sync`)）：一个同步调度器，用于调试
- 分布式：一个分布式调度器，用于在多台机器上执行图，见下文。

要选择其中之一进行计算，您可以在请求结果时指定，例如
```python
myvalue.compute(scheduler="single-threaded")  # 用于调试
```

您也可以临时地
```python
with dask.config.set(scheduler='processes'):
    # 仅为此块临时设置
    # 此块中的所有计算调用都将使用指定的调度器
    myvalue.compute()
    anothervalue.compute()
```

或全局地
```python
# 设置直至另行通知
dask.config.set(scheduler='processes')
```
设置默认调度器。

让我们在熟悉的航班数据案例上尝试一些调度器。

In [ ]:
%run prep.py -d flights

In [ ]:
import dask.dataframe as dd
import os
df = dd.read_csv(os.path.join('data', 'nycflights', '*.csv'),
                 parse_dates={'Date': [0, 1, 2]},
                 dtype={'TailNum': object,
                        'CRSElapsedTime': float,
                        'Cancelled': bool})

# 按始发机场分组的最大平均未取消延误
largest_delay = df[~df["Cancelled"]].groupby('Origin').DepDelay.mean().max()
largest_delay

In [ ]:
# 以下每个都给出相同的结果（您可以检查！）
# 有什么惊喜吗？
import time
for sch in ['threading', 'processes', 'sync']:
    t0 = time.time()
    r = largest_delay.compute(scheduler=sch)
    t1 = time.time()
    print(f"{sch:>10}, {t1 - t0:0.4f} s; result, {r:0.2f} hours")

### 需要思考的一些问题:
- 此任务可能有多大的加速（提示，请查看任务图）？
- 考虑到这台机器上的内核数量，并行调度器可以比单线程调度器快多少？
- 使用线程比单线程快多少？ 为什么这与最佳加速不同？
- 为什么这里的多进程调度器慢这么多？

`多线程` 调度器是在单台机器上处理超过内存大小的大型数据集的不错选择，只要所使用的函数大多是时候都释放 [全局解释器(GIL)](https://wiki.python.org/moin/GlobalInterpreterLock)。 NumPy 和 Pandas 在大多数地方都释放了 GIL，因此 `threaded` 调度程序是 `dask.array` 和 `dask.dataframe` 的默认设置。 分布式调度程序，也许有了`processes=False`，也可以在单台机器上很好地处理这些工作负载。

对于保持 GIL 的工作负载（常见于 `dask.bag` 和用 `dask.delayed` 包装的自定义代码），我们建议使用分布式调度程序，即使在单台机器上也是如此。 一般来说，它比`进程`调度器更智能并提供更好的诊断。

[https://docs.dask.org/en/latest/scheduling.html](https://docs.dask.org/en/latest/scheduling.html) 提供有关选择调度器的一些其他详细信息.

跨集群扩展工作需要分布式调度程序。

## 创建一个集群

### 简单的方法

`dask.distributed` 系统由单个中心化的调度器(`scheduler`)和一个或多个工作进程(`worker`)组成。 [部署](https://docs.dask.org/en/latest/setup.html) 远程 Dask 集群需要一些额外的工作。 但是在本地做事只需要创建一个`Client` 对象，它可以让你与"集群(`cluster`)"（你机器上的本地线程或进程）交互。 有关更多信息，请参阅 [此处](https://docs.dask.org/en/latest/setup/single-distributed.html)。

注意`Client()`需要很多可选的[参数](https://distributed.dask.org/en/latest/local-cluster.html#api)，来配置进程/线程数、内存限制和其他配置。

In [ ]:
from dask.distributed import Client

# 设置本地集群
# 默认情况下，这会为每个核心设置1个worker
client = Client()
client.cluster

如果您并非在jupyterlab中使用"dask-labextension"，请务必单击"仪表板(`daskboard`)"链接以打开诊断仪表板。

## 使用分布式客户端执行

考虑一些琐碎的计算(例如我们之前使用过的那些)，我们添加了 sleep 语句以模拟正在完成的实际工作。

In [ ]:
from dask import delayed
import time

def inc(x):
    time.sleep(5)
    return x + 1

def dec(x):
    time.sleep(3)
    return x - 1

def add(x, y):
    time.sleep(7)
    return x + y

默认情况下，创建一个`Client`会使其成为默认调度程序。 任何对 `.compute` 的调用都将使用你的 `client` 附加到的集群，除非你另外指定（如上文所述）。

In [ ]:
x = delayed(inc)(1)
y = delayed(dec)(2)
total = delayed(add)(x, y)
total.compute()

任务将在集群处理时出现在Web UI中，最终，结果将作为上述单元格的输出打印出来。 请注意，内核在等待结果时被阻塞。 生成的任务块图可能如下所示。 将鼠标悬停在每个块上会给出它相关的功能，以及执行所需的时间。 ![这里](images/tasks.png)

只要计算正在进行中，您还可以在仪表板的"任务图(Graph)"窗格中看到正在执行的任务图的简化版本。

让我们回到之前的航班计算，看看仪表板上发生了什么（您可能希望notebook和dashboard并排放置）。 与之前相比，这表现如何？

In [ ]:
%time largest_delay.compute()

在这种特殊情况下，这应该与上面的最佳情况线程处理一样快或更快。 你认为这是为什么？ 你应该开始阅读[这里](https://distributed.dask.org/en/latest/index.html#architecture)，特别要注意分布式调度器是一个完全重写的版本，在中间结果共享以及哪些任务在哪个worker上运行等方面具有更多智能。 这将在*某些*情况下带来更好的性能，但与线程调度器相比仍然有更大的延迟和开销，因此在极少数情况下它的性能更差。 幸运的是，仪表板现在为我们提供了更多 [诊断信息](https://distributed.dask.org/en/latest/diagnosing-performance.html)。 查看仪表板的 Profile 页面，找出我们刚刚执行的计算占用 CPU 时间最多的部分是什么？

如果您只想执行使用delayed创建的计算，或者运行基于更高级别数据集合的计算，那么这就是将工作扩展到集群规模所需的全部内容。 但是，有更多关于分布式调度程序的详细信息将有助于有效使用。 请参阅[进阶分布式](06_distributed_advanced.ipynb)一章。

### 练习

在查看诊断页面的同时运行以下计算。 在每种情况下，什么花费的时间最多？

In [ ]:
# 航班数
_ = len(df)

In [ ]:
# 未取消的航班数
_ = len(df[~df.Cancelled])

In [ ]:
# 每个机场的未取消航班数
_ = df[~df.Cancelled].groupby('Origin').Origin.count().compute()

In [ ]:
# 每个机场的平均起飞延误时间？
_ = df[~df.Cancelled].groupby('Origin').DepDelay.mean().compute()

In [ ]:
# 每day-of-week平均出发延误
_ = df.groupby(df.Date.dt.dayofweek).DepDelay.mean().compute()

In [ ]:
client.shutdown()